In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, make_scorer

pd.set_option('display.max_columns', None)

In [2]:
# Import dataset
df = pd.read_csv("train.csv")

In [3]:
# Make a pipeline to prepare data 
from sklearn.base import BaseEstimator, TransformerMixin

class FillNanValues(BaseEstimator, TransformerMixin):
    """
    Gets a DataFrame describing houses and fill the NaNs with meaningful replacements
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.LotFrontageMedian = X.LotFrontage.median()
        self.MasVnrAreaMedian = X.MasVnrArea.median()
        return self
    def transform(self, X):
        X.LotFrontage.fillna(self.LotFrontageMedian, inplace=True)
        X.MasVnrArea.fillna(self.MasVnrAreaMedian, inplace=True)
        X.MiscFeature.fillna("NoShed", inplace=True)
        X.Fence.fillna('NoFence', inplace=True)
        X.BsmtQual.fillna("NoBasement", inplace=True)
        X.BsmtCond.fillna("NoBasement", inplace=True)
        X.BsmtExposure.fillna("NoBasement", inplace=True)
        X.BsmtFinType1.fillna("NoBasement", inplace=True)
        X.BsmtFinType2.fillna("NoBasement", inplace=True)
        X.BsmtFinType2.fillna("NoBasement", inplace=True)
        X.MasVnrType.fillna("None", inplace=True)
        X.Electrical.fillna("SBrkr", inplace=True)
        X.FireplaceQu.fillna('NoFP', inplace=True)
        X.GarageType.fillna('Attchd', inplace=True)
        X.GarageFinish.fillna('Unf', inplace=True)
        X.GarageQual.fillna('TA', inplace=True)
        X.GarageCond.fillna('TA', inplace=True)
        X.PoolQC.fillna("NoPool", inplace=True)
        return X

class RemoveColumns(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and removes irrelevant, repetative, or corrupt data
    Decisions based on EDA found in ML_Final_EDA.ipynb
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(
        ['GarageArea', 'BsmtHalfBath', 'MiscVal', 'MoSold',
        'YrSold', 'BsmtFinSF2', 'LowQualFinSF', '3SsnPorch', 
        'Utilities', 'Exterior2nd', 'Alley', 'Id'],
        axis = 1)

class AlterAttributes(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and alter some attributes 
    Decisions based on EDA found in ML_Final_EDA.ipynb
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        #Combine GarageCars == 3 and GarageCars == 4
        X.GarageCars = X.GarageCars.map(lambda x: 3 if (x == 4) else x)

        #Fill Nans in GarageYrBlt with median year
        X.GarageYrBlt = X.GarageYrBlt.fillna(X.GarageYrBlt.median())

        #Combine KitchenAbvGr 1 and 0 AND 2 and 3 such that it is binary
        X.KitchenAbvGr = X.KitchenAbvGr.map(lambda x: 0 if (x == 1) else x)
        X.KitchenAbvGr = X.KitchenAbvGr.map(lambda x: 1 if (x == 2 or x == 3) else x)

        #Make Pool Area binary (x>0 or x==0)--> 1 if they have pool else 0
        X.PoolArea = X.PoolArea.map(lambda x: 1 if (x>0) else x)
        return X

class GetDummies(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and convert all categorical attributes to one-hot encoded columns
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.PriorData = pd.get_dummies(pd.DataFrame(X))
        self.columns = self.PriorData.columns
        return self
    def transform(self, X):
        temp = pd.get_dummies(X)
        for col in self.columns:
            if col not in temp.columns:
                temp_ar = np.zeros(temp.shape[0])
                for i in range(len(temp_ar)):
                    temp_ar[i]=self.PriorData[col].median()
                temp[col] = temp_ar
                
        for i,j in zip(temp.isnull().sum(), temp.columns):
            if i>0:
                temp[j].fillna(temp[j].median(), inplace=True)
        return pd.get_dummies(temp)

class ScaleData(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame, center & scale it using the mean and standard devision 
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.PriorData = X
        return self
    def transform(self, X):
        return (X-self.PriorData.mean())/self.PriorData.std()    

class GetDataFrame(BaseEstimator, TransformerMixin):
    """
    Get any data object and convert it to a Pandas DataFrame
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.columns = pd.DataFrame(X).columns
        return self
    def transform(self, X):
        return pd.DataFrame(X, columns=self.columns).fillna(0)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

Prep = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame())
])

In [4]:
# Test pipeline
Prep.fit_transform(df).head(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBasement,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NoBasement,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBasement,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBasement,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBasement,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFP,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_NoPool,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFence,MiscFeature_Gar2,MiscFeature_NoShed,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,Sal

In [5]:
Bag_kNN = Pipeline([
    ('prep', Prep), 
    ('bag', BaggingRegressor(KNeighborsRegressor(n_neighbors=7, weights='distance'), n_estimators=100))
])

RandFor = Pipeline([
    ('prep', Prep), 
    ('bag', RandomForestRegressor(n_estimators=100))
])

ExTree = Pipeline([
    ('prep', Prep), 
    ('bag', ExtraTreesRegressor(n_estimators=100))
])

AdaB = Pipeline([
    ('prep', Prep), 
    ('bag', AdaBoostRegressor(n_estimators=100))
])

GradB = Pipeline([
    ('prep', Prep), 
    ('bag', GradientBoostingRegressor(n_estimators=100))
])

Models = [
    ("Bag kNN", Bag_kNN),
    ('RandFor', RandFor), 
    ('ExTrees', ExTree), 
    ('AdaB', AdaB),
    ('GradB', GradB)
]

In [6]:
for name, model in Models:
    print name
    print '\t', cross_val_score(model, df.drop("SalePrice", axis=1), df.SalePrice, cv=3, scoring=make_scorer(mean_squared_log_error))

Bag kNN
	

/Users/brendangarrett/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


[ 0.03808122  0.04500383  0.03517586]
RandFor
	[ 0.01953026  0.02404002  0.02135008]
ExTrees
	[ 0.01875796  0.02671833  0.02055463]
AdaB
	[ 0.03715334  0.04338534  0.04048926]
GradB
	[ 0.01447778  0.02151079  0.01800248]


#### Random Forest, ExtraTrees, and GradientBoosting seem to be the best models with default hyperparams
 Do GridSearchCV to find better hyperparams

In [7]:
param_grid = [
    {'bag__n_estimators' : [1,10, 50, 100, 500, 1000]}
]

grid_search = GridSearchCV(ExTree, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search.cv_results_["mean_test_score"],grid_search.cv_results_["params"])

[(0.047740336191483171, {'bag__n_estimators': 1}),
 (0.02393405988683844, {'bag__n_estimators': 10}),
 (0.022121865637518653, {'bag__n_estimators': 50}),
 (0.021762093803667878, {'bag__n_estimators': 100}),
 (0.021780278826132968, {'bag__n_estimators': 500}),
 (0.02145741822352824, {'bag__n_estimators': 1000})]

In [8]:
param_grid = [
    {'bag__n_estimators' : [1,10, 50, 100, 500, 1000]}
]

grid_search2 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search2.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search2.cv_results_["mean_test_score"],grid_search.cv_results_["params"])

[(0.14494506715327282, {'bag__n_estimators': 1}),
 (0.059736048957318495, {'bag__n_estimators': 10}),
 (0.02087601225108222, {'bag__n_estimators': 50}),
 (0.018061917534497974, {'bag__n_estimators': 100}),
 (0.01706036620161459, {'bag__n_estimators': 500}),
 (0.016899411824606601, {'bag__n_estimators': 1000})]

In [9]:
param_grid = [
    {'bag__n_estimators' : [1,10, 50, 100, 500, 1000]}
]

grid_search3 = GridSearchCV(RandFor, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search3.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search3.cv_results_["mean_test_score"],grid_search3.cv_results_["params"])

[(0.048641785569200797, {'bag__n_estimators': 1}),
 (0.024085429180628622, {'bag__n_estimators': 10}),
 (0.022067027734508384, {'bag__n_estimators': 50}),
 (0.021851462136995282, {'bag__n_estimators': 100}),
 (0.021256549050969589, {'bag__n_estimators': 500}),
 (0.021329105582525914, {'bag__n_estimators': 1000})]

## GradientBoostingRegressor with 1000 estimators seems to work the best so far

In [10]:
param_grid = [
    {'bag__n_estimators' : [500, 800, 1000]},
    {'bag__loss' : ['ls', 'lad', 'huber']},
    {'bag__learning_rate' : [0.01, 0.05, 0.1, 0.2]},
    {'bag__max_depth' : [2,3,5,7]},
    {'bag__subsample' : [0.7, 1.0]},
    {'bag__max_features' : [None, 'sqrt']}
]

grid_search2 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search2.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search2.cv_results_["mean_test_score"],grid_search2.cv_results_["params"])

[(0.017290105055515374, {'bag__n_estimators': 500}),
 (0.016732352766438981, {'bag__n_estimators': 800}),
 (0.016867120767244677, {'bag__n_estimators': 1000}),
 (0.018055217259151806, {'bag__loss': 'ls'}),
 (0.019296993254005759, {'bag__loss': 'lad'}),
 (0.01799374553998026, {'bag__loss': 'huber'}),
 (0.061302260409518326, {'bag__learning_rate': 0.01}),
 (0.020656818851647798, {'bag__learning_rate': 0.05}),
 (0.018175624146071028, {'bag__learning_rate': 0.1}),
 (0.018535018422424415, {'bag__learning_rate': 0.2}),
 (0.019777090415792814, {'bag__max_depth': 2}),
 (0.017983112170276854, {'bag__max_depth': 3}),
 (0.018373101041465157, {'bag__max_depth': 5}),
 (0.019948600431825978, {'bag__max_depth': 7}),
 (0.017381928386113524, {'bag__subsample': 0.7}),
 (0.017952341434073083, {'bag__subsample': 1.0}),
 (0.018034749758103395, {'bag__max_features': None}),
 (0.019675806264996629, {'bag__max_features': 'sqrt'})]

In [11]:
model = Pipeline([
    ('prep', Prep), 
    ('bag', GradientBoostingRegressor(n_estimators=800, loss='huber', subsample=0.7))
])

r = cross_val_score(model, df.drop("SalePrice", axis=1), df.SalePrice, cv=3, scoring=make_scorer(mean_squared_log_error))
print r, r.mean()

[ 0.01393633  0.02140258  0.0162505 ] 0.0171964703662


# Try it on the test data and make a submission to Kaggle

In [12]:
# Train on entire training dataset 
model.fit(df.drop(['SalePrice'],axis=1), df.SalePrice)

df_test = pd.read_csv("test.csv")
test_predictions = model.predict(df_test)

pd.DataFrame(test_predictions).describe()

,0
count,1459.000000
mean,178766.736494
std,77413.739005
min,21586.827926
25%,128701.563174
50%,158293.934190
75%,209288.495966
max,546162.750549


In [13]:
df_test_predictions = pd.DataFrame([df_test.Id.astype(int) , test_predictions]).T
df_test_predictions.columns = ['Id', 'SalePrice']
df_test_predictions['Id'] =df_test_predictions['Id'].astype(int)
df_test_predictions.to_csv("Submission_2.csv", index=False)
df_test_predictions

,Id,SalePrice
0,1461,115477.550193
1,1462,161452.414532
2,1463,194524.527108
3,1464,196959.246513
4,1465,179670.196399
5,1466,175468.544157
6,1467,168488.819657
7,1468,170201.114119
8,1469,187439.701625
9,1470,129996.448591
